# Strange Attractor Dataset Generation Demo

This notebook demonstrates how to create chaotic datasets following the "Idiot's Guide" methodology.

In [ ]:
import sys
import os
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from strange_attractor_generator import LorenzGenerator, TimeDelayEmbedding, DatasetManager

## Part 1: Lorenz System Dataset

In [ ]:
# Create Lorenz generator
lorenz = LorenzGenerator()

# Generate single trajectory
data = lorenz.generate_single_trajectory()
print(f"Generated {len(data)} points")
data.head()

In [ ]:
# Plot the famous butterfly attractor
fig = plt.figure(figsize=(12, 4))

# 2D projections
ax1 = fig.add_subplot(131)
ax1.plot(data['x'], data['z'], linewidth=0.5)
ax1.set_xlabel('x')
ax1.set_ylabel('z')
ax1.set_title('x-z projection')

ax2 = fig.add_subplot(132)
ax2.plot(data['x'], data['y'], linewidth=0.5)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_title('x-y projection')

# 3D plot
ax3 = fig.add_subplot(133, projection='3d')
ax3.plot(data['x'], data['y'], data['z'], linewidth=0.5)
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('z')
ax3.set_title('3D Attractor')

plt.tight_layout()
plt.show()

## Part 2: Parameter Sweep Dataset

In [ ]:
# Generate parameter sweep - varying rho from stable to chaotic
rho_values = np.linspace(10, 40, 8)  # Fewer values for demo
sweep_data = lorenz.generate_parameter_sweep('rho', rho_values, n_points=2000)

print(f"Generated {len(sweep_data)} total points")
print(f"Parameter values: {rho_values}")
sweep_data.groupby('rho').size()

In [ ]:
# Plot multiple attractors for different rho values
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, rho in enumerate(rho_values):
    subset = sweep_data[sweep_data['rho'] == rho]
    axes[i].plot(subset['x'], subset['z'], linewidth=0.5)
    axes[i].set_title(f'ρ = {rho:.1f}')
    axes[i].set_xlabel('x')
    axes[i].set_ylabel('z')

plt.suptitle('Lorenz Attractor for Different ρ Values')
plt.tight_layout()
plt.show()

## Part 3: Time-Delay Embedding

In [ ]:
# Create sample signal (noisy sine wave)
embedding = TimeDelayEmbedding()
t, signal = embedding.create_sample_signal(duration=5.0, frequency=3.0)

# Plot original signal
plt.figure(figsize=(10, 3))
plt.plot(t, signal)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title('Original 1D Signal')
plt.show()

In [ ]:
# Perform time-delay embedding
tau = 25  # time delay
m = 3     # embedding dimension

embedded = embedding.embed_signal(signal, tau, m)
print(f"Original signal length: {len(signal)}")
print(f"Embedded coordinates shape: {embedded.shape}")

In [ ]:
# Plot embedded attractor
fig = plt.figure(figsize=(12, 4))

# 2D projection
ax1 = fig.add_subplot(121)
ax1.scatter(embedded[:, 0], embedded[:, 1], s=1, alpha=0.7)
ax1.set_xlabel('x(t)')
ax1.set_ylabel('x(t+τ)')
ax1.set_title('2D Embedded Attractor')

# 3D plot
ax2 = fig.add_subplot(122, projection='3d')
ax2.scatter(embedded[:, 0], embedded[:, 1], embedded[:, 2], s=1, alpha=0.7)
ax2.set_xlabel('x(t)')
ax2.set_ylabel('x(t+τ)')
ax2.set_zlabel('x(t+2τ)')
ax2.set_title('3D Embedded Attractor')

plt.tight_layout()
plt.show()

## Saving Datasets

In [ ]:
# Initialize dataset manager
dataset_mgr = DatasetManager()

# Save datasets with metadata
dataset_mgr.save_dataset(
    data,
    "demo_lorenz_single",
    {
        "description": "Demo Lorenz trajectory",
        "parameters": {"sigma": 10.0, "rho": 28.0, "beta": 8.0/3.0},
        "n_points": len(data)
    }
)

# Convert embedded coordinates to DataFrame
embedded_df = pd.DataFrame(embedded, columns=['x1', 'x2', 'x3'])
dataset_mgr.save_dataset(
    embedded_df,
    "demo_embedded_signal",
    {
        "description": "Demo embedded sine wave",
        "embedding_params": {"tau": tau, "dimension": m},
        "n_points": len(embedded_df)
    }
)

print("Demo datasets saved!")